为了在训练过程中添加进度条，你可以使用 `tqdm` 这个库。它是一个快速、可扩展的Python进度条，可以在 Python 循环或迭代过程中提供美观的进度显示。

### 安装 `tqdm`

首先确保你已经安装了 `tqdm`：

```bash
pip install tqdm
```

### 在训练循环中引入 `tqdm`

接下来，你需要修改你的训练函数来集成 `tqdm`。以下是改进后的代码示例，展示了如何将 `tqdm` 集成到你的 `fit_gpu` 函数中：

```python
from tqdm import tqdm

def fit_gpu(Train_DL, TST_DL, Model_m, epoch_, optim, loss_fn):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    Model_m.to(device)

    correct = 0
    total = 0
    running_loss = 0

    Model_m.train()
    for x, y in tqdm(Train_DL, desc=f'Epoch {epoch_ + 1}/{epochs} Training', unit='batch'):
        x = x.to(device)
        y = y.to(device)

        y_pred = Model_m(x)
        loss = loss_fn(y_pred, y)

        optim.zero_grad()
        loss.backward()
        optim.step()

        with torch.no_grad():
            predicted = torch.argmax(y_pred, dim=1)
            correct += (predicted == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()

    epoch_acc = correct / total
    epoch_loss = running_loss / len(Train_DL.dataset)

    # 测试阶段
    Model_m.eval()
    tstcorrect = 0
    tsttotal = 0
    tstrunning_loss = 0

    with torch.no_grad():
        for x, y in tqdm(TST_DL, desc=f'Epoch {epoch_ + 1}/{epochs} Testing', unit='batch'):
            x, y = x.to(device), y.to(device)
            y_pred = Model_m(x)
            loss = loss_fn(y_pred, y)

            predicted = torch.argmax(y_pred, dim=1)
            tstcorrect += (predicted == y).sum().item()
            tsttotal += y.size(0)
            tstrunning_loss += loss.item()

    tstepoch_acc = tstcorrect / tsttotal
    tstepoch_loss = tstrunning_loss / len(TST_DL.dataset)

    print(
        f'epoch:{epoch_} | loss:{epoch_loss:.3f} | acc:{epoch_acc:.3f} | tstloss:{tstepoch_loss:.3f} | tstacc:{tstepoch_acc:.3f}'
    )

    return epoch_loss, epoch_acc, tstepoch_loss, tstepoch_acc
```

### 代码解释

- **导入 `tqdm`**：从 `tqdm` 库中导入 `tqdm`。
- **包裹数据加载器（DataLoader）**：使用 `tqdm()` 包裹 `Train_DL` 和 `TST_DL`，这样每次迭代都会更新进度条。
- **设置描述和单位**：通过 `desc` 参数设置进度条的描述信息，`unit='batch'` 设置每个单元为 batch。

### 输出效果

当你运行训练脚本时，你会看到类似下面的输出：

```
Epoch 1/10 Training: 100%|██████████| 938/938 [00:15<00:00, 62.47batch/s]
Epoch 1/10 Testing:  100%|██████████| 156/156 [00:02<00:00, 77.90batch/s]
epoch:0 | loss:0.002 | acc:0.951 | tstloss:0.007 | tstacc:0.888
...
```

这不仅使你能够直观地了解训练进度，还能提升调试和观察训练过程的效率。

希望这能帮助你在训练过程中更好地监控进度！如果你有其他问题或需要进一步的帮助，请随时告诉我。 😊